In [21]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [22]:
with open('all_children.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


In [61]:
def getChildInfo(number):
    # URL сторінки
    url = f"https://www.msp.gov.ua/children/child.php?n={number}"

    # Виконання HTTP-запиту
    response = requests.get(url)

    # Перевірка статусу відповіді
    if response.status_code == 200:
        # get object soup
        soup = BeautifulSoup(response.content, 'html.parser')
        name, age = "no name", "no age"
        name_age = soup.find(class_='passport__title-name').text #get name & age
        name = name_age.split(',')[0] #get name
        age = (name_age.split(' ')[1]) #get age
        child_id = soup.find(class_='passport__title-number').text.split('№')[1] #get id

        # get block passport__content with information with child
        passport_content = soup.find('div', class_='row passport__content')
        gender = "uknown"
        gender_element = passport_content.find('span', class_='passport__content-title', string="Стать:")
        if gender_element:
            gender = gender_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Стать не знайдено.")
        
        # get region
        region = "uknown"
        region_element = passport_content.find('span', class_='passport__content-title', string="Область:")
        if region_element:
            region = region_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Область не знайдено.")
        
        #get disablement status
        disablement = "uknown"
        disablement_element = passport_content.find('span', class_='passport__content-title', string="Наявність інвалідності:")
        if disablement_element:
            disablement = disablement_element.find_next('span', class_='passport__content-inner').text.strip()
            if disablement == 'є':
                disablement = True
            else:
                disablement = False

        else:
            print("Статус інвалідності не знайдено")

        #get form of adoption
        adoptionForm_element = passport_content.find('span', class_='passport__content-title', string="Можлива форма влаштування:")
        if adoptionForm_element:
            adoptionForm = adoptionForm_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Форому всиновлення не знайдено.")

        #place where child is now
        place_element = "uknown"
        place_element = passport_content.find('span', class_='passport__content-title', string="Куди звертатись:")
        if place_element:
            place = place_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Місце перебування не знайдено.")

        #photo
        photo_element = passport_content.find('a', class_='js-lightBoxGallery')
        if photo_element:
            photo = 'https://www.msp.gov.ua/'+photo_element.find('img').get('src')
        else:
            print("Фотокартка не знайдена.")
        
        #get list of siblings, their names and links to their pages


        # #siblings
        siblings_count = 0
        siblings_element = passport_content.find('ul', class_='passport__content-inner passport__content-inner--brother')
        if siblings_element:
            siblings_count = len(siblings_element.findAll('li'))            
            siblings_list_element = siblings_element.findAll('a')
            siblings_list = []
            if len(siblings_list_element) > 0:
                for i in siblings_list_element:
                    sibling = "https://www.msp.gov.ua" + i.attrs['href']
                    siblings_list.append(sibling)
            else:
                siblings_list = ["no siblings"]
                print(siblings_list)
        else:
             print("Братів не знайдено")
             siblings_list = ["no siblings"]

            
        print(url,'\nname:',name, '\nage:',age, '\nid:',child_id, '\ngender:', gender, '\nregion: ',region,
               '\ndisablement: ', disablement, "\nadoptionForm: ", adoptionForm, "\nplace: ", place, "\nphoto url:", photo,
                '\nsiblings_count:', siblings_count, '\n*******\nsiblings_list', siblings_list,
              '\n--------------------------------------------------','\n--------------------------------------------------')
        
        
    
    else:
        print(f"Не вдалося отримати сторінку, статус-код: {response.status_code}")

#перевірка функціоналу
for i in range(1,100):
     getChildInfo(data[i])
    # getChildInfo('388104')

https://www.msp.gov.ua/children/child.php?n=6829 
name: Валентин 
age: 16 
id: 6829 
gender: чоловіча 
region:  Вінницька обл. 
disablement:  False 
adoptionForm:  національне усиновлення; 
place:  ССД Жмеринської райдержадміністрації: 04332  5-00-50 
photo url: https://www.msp.gov.ua/../files/children/02_children18.01.2025.zip/36367557.jpeg 
siblings_count: 2 
*******
siblings_list ['https://www.msp.gov.ua/children/child.php?n=6831', 'https://www.msp.gov.ua/children/child.php?n=169303'] 
-------------------------------------------------- 
--------------------------------------------------
https://www.msp.gov.ua/children/child.php?n=6831 
name: Ірина 
age: 15 
id: 6831 
gender: жіноча 
region:  Вінницька обл. 
disablement:  False 
adoptionForm:  національне усиновлення; опіка/піклування; прийомна сім'я та дитячий будинок сімейного типу; міждержавне усиновлення; 
place:  ССД Жмеринської райдержадміністрації: 04332  5-00-50; Національна соціальна сервісна служба України Департамент з пит